In [1]:
import os
import sys
sys.path.append("/users/ziyzhang/topology-research")
from topologies.DDF import DDFtopo
from nexullance.MD_Nexullance_MP.MD_Nexullance_MP import MD_Nexullance_MP
from nexullance.Nexullance_MP import Nexullance_MP
sys.path.append("/users/ziyzhang/topology-research/nexullance/IT_boost/build")
from Nexullance_IT_cpp import MD_Nexullance_IT_interface
import globals as gl
import numpy as np
import pickle
import csv
import time
import tracemalloc
import pandas as pd



In [2]:
Cap_remote = 10 #GBps
Cap_local = 10 #GBps

config = gl.ddf_configs[0]
V = config[0]
D = config[1]
EPR = (D+1)//2
_network = DDFtopo(V, D)
arcs = _network.generate_graph_arcs()

ASP, _ = _network.calculate_all_shortest_paths()
ECMP_ASP = gl.ECMP(ASP)

M_Rs = []
M_R_names = []
M_EPs = []
max_local_link_loads = []

# # # Define multiple traffic demand matrices:
# # shifts
for _shift in range(1, V*EPR):
# for _shift in range(1, 5):
    M_EP = gl.generate_shift_traffic_pattern(V, EPR, _shift)
    # try to scale the traffic scaling factor to 10x saturation under ECMP_ASP
    remote_link_flows, local_link_flows = _network.distribute_M_EPs_on_weighted_paths(ECMP_ASP, EPR, M_EP)
    max_remote_link_load = np.max(remote_link_flows)/Cap_remote
    max_local_link_load = np.max(local_link_flows)/Cap_local
    traffic_scaling = 10.0/max(max_local_link_load, max_remote_link_load)
    M_EP = traffic_scaling * M_EP
    M_R = gl.convert_M_EPs_to_M_R(M_EP, V, EPR)
    # ==============
    # manage data
    M_EPs.append(M_EP)
    M_Rs.append(M_R)
    max_local_link_loads.append(max_local_link_load)
    M_R_names.append(f"shift_{_shift}")
M_R_weights = [1/len(M_Rs) for _ in range(len(M_Rs))]


In [3]:
def do_MD_IT(alpha:float, beta:float) -> float:
    # calculate Phi for MD_Nexullance_IT routing
    md_nexu_it = MD_Nexullance_IT_interface(V, arcs, M_Rs, M_R_weights, False)
    md_nexu_it.set_parameters(alpha, beta)
    
    tracemalloc.start()
    start_time = time.time()
    md_nexu_it.run()
    end_time = time.time()
    MD_peak_RAM = tracemalloc.get_traced_memory()[1]/1024/1024
    MD_time = end_time-start_time
    tracemalloc.stop()

    return -1*md_nexu_it.get_weighted_max_link_load()

In [4]:
from bayes_opt import BayesianOptimization
# Bounded region of parameter space
pbounds = {'alpha': (0.1, 50.0), 'beta': (0.0, 50.0)}
optimizer = BayesianOptimization(
    f=do_MD_IT,
    pbounds=pbounds,
    random_state=1,
)
optimizer.probe(
    params=[0.1, 7.0],
    lazy=True,
)
optimizer.probe(
    params=[20, 0.1],
    lazy=True,
)

: 

In [5]:
optimizer.maximize(
    init_points=10,
    n_iter=200,
)

|   iter    |  target   |   alpha   |   beta    |
-------------------------------------------------
| 1         | -5.531    | 0.1       | 7.0       |
| 2         | -10.0     | 20.0      | 0.1       |
| 3         | -5.581    | 20.91     | 36.02     |
| 4         | -5.629    | 0.1057    | 15.12     |
| 5         | -5.619    | 7.423     | 4.617     |
| 6         | -5.609    | 9.394     | 17.28     |
| 7         | -5.568    | 19.9      | 26.94     |
| 8         | -5.636    | 21.02     | 34.26     |
| 9         | -5.619    | 10.3      | 43.91     |
| 10        | -5.635    | 1.467     | 33.52     |
| 11        | -5.636    | 20.92     | 27.93     |
| 12        | -5.526    | 7.105     | 9.905     |
| 13        | -5.64     | 44.46     | 50.0      |
| 14        | -5.569    | 50.0      | 28.99     |
| 15        | -5.638    | 27.94     | 50.0      |
| 16        | -5.611    | 50.0      | 10.33     |


In [ ]:
optimizer.max

{'target': -2.8341131070711123,
 'params': {'alpha': 4.6419767269134455, 'beta': 6.593203706205871}}